In [6]:
#sleep deprivation go brrrrr

In [42]:
import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [131]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [137]:
default_blacklist=['_','/','#','.',',','?','[',']','-',':','@','"',"'",'_','0','1','2','3','4','5','6','7','8','9']

def pd_tokenize(pandas_series, blacklist=default_blacklist):
    tokenized=[]
    for _ in pandas_series:
        for chara in blacklist:
            _=_.replace(chara,'')
        tokens=word_tokenize(_.lower())
        tokenized.append(tokens)
    return(pd.Series(tokenized))

def pd_clean(pandas_series, blacklist=default_blacklist):
    tokenized=[]
    for _ in pandas_series:
        for chara in blacklist:
            _=_.replace(chara,'')
        tokens=word_tokenize(_.lower())
        tokenized.append(' '.join(tokens))
    return(pd.Series(tokenized))

In [138]:
tokens=pd_tokenize(train.text)
cleaned=pd_clean(train.text)
train['tokens']=tokens
train['cleaned']=cleaned

In [139]:
train

,id,keyword,location,text,target,tokens,cleaned
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[our, deeds, are, the, reason, of, this, earth...",our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[all, residents, asked, to, shelter, in, place...",all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order...",people receive wildfires evacuation orders in ...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[just, got, sent, this, photo, from, ruby, ala...",just got sent this photo from ruby alaska as s...
...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,"[two, giant, cranes, holding, a, bridge, colla...",two giant cranes holding a bridge collapse int...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,"[ariaahrary, thetawniest, the, out, of, contro...",ariaahrary thetawniest the out of control wild...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"[m, utckm, s, of, volcano, hawaii, httptcozdto...",m utckm s of volcano hawaii httptcozdtoydebj
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,"[police, investigating, after, an, ebike, coll...",police investigating after an ebike collided w...


In [140]:
train0=train[train['target']==0].copy()
train1=train[train['target']==1].copy()
train0.reset_index(inplace=True)
train1.reset_index(inplace=True)

# CountVectorizer

In [155]:
cv0=CountVectorizer(stop_words='english')
cv1=CountVectorizer(stop_words='english')

xcv0=cv0.fit_transform(train0.cleaned)
xcv1=cv1.fit_transform(train1.cleaned)

xcva0=xcv0.toarray()
xcva1=xcv1.toarray()

In [163]:
tweet_series_0=pd.Series(name="count", data=0, index=cv0.get_feature_names())
for tnum in range(0,len(xcva0)):
    s=pd.Series(name="Count 0", data=xcva0[tnum], index=cv0.get_feature_names())
    s=s[s>0]
    s=pd.DataFrame(s)
    s.reset_index(inplace=True)
    for il in range(len(s)):
        tweet_series_0.at[s['index'].iloc[il]]+=1

        
tweet_series_1=pd.Series(name="count", data=0, index=cv1.get_feature_names())
for tnum in range(0,len(xcva1)):
    s=pd.Series(name="Count 1", data=xcva1[tnum], index=cv1.get_feature_names())
    s=s[s>0]
    s=pd.DataFrame(s)
    s.reset_index(inplace=True)
    for il in range(len(s)):
        tweet_series_1.at[s['index'].iloc[il]]+=1

In [143]:
word_blacklist=['http','https']

In [180]:
ts0=tweet_series_0.copy()
ts0.sort_values(ascending=False, inplace=True)
ts0=pd.DataFrame(ts0)
ts0.reset_index(inplace=True)

ts1=tweet_series_1.copy()
ts1.sort_values(ascending=False, inplace=True)
ts1=pd.DataFrame(ts1)
ts1.reset_index(inplace=True)

f=open('disaster_1000_words.csv','w')
f.write('word,count,percent_frequency')
for _ in range(1000):
    f.write(ts1.iloc[_]['index']+','+str(ts1.iloc[_]['count'])+','+str(ts1.iloc[_]['count']/len(ts1)*100)[0:6]+'%,\n')
f.close()

f=open('safe_1000_words.csv','w')
f.write('word,count,percent_frequency')
for _ in range(1000):
    f.write(ts0.iloc[_]['index']+','+str(ts0.iloc[_]['count'])+','+str(ts0.iloc[_]['count']/len(ts0)*100)[0:6]+'%,\n')

f.close()

# TFIDF


In [177]:
tf0=TfidfVectorizer(stop_words='english')
tf1=TfidfVectorizer(stop_words='english')

xtf0=tf0.fit_transform(train0.cleaned)
xtf1=tf1.fit_transform(train1.cleaned)

xtfa0=xtf0.toarray()
xtfa1=xtf1.toarray()

In [178]:
tweet_series_0_tf=pd.Series(name="count", data=0, index=tf0.get_feature_names())
for tnum in range(0,len(xtfa0)):
    s=pd.Series(name="Count 0", data=xtfa0[tnum], index=tf0.get_feature_names())
    s=s[s>0]
    s=pd.DataFrame(s)
    s.reset_index(inplace=True)
    for il in range(len(s)):
        tweet_series_0_tf.at[s['index'].iloc[il]]+=1

        
tweet_series_1_tf=pd.Series(name="count", data=0, index=tf1.get_feature_names())
for tnum in range(0,len(xtfa1)):
    s=pd.Series(name="Count 1", data=xtfa1[tnum], index=tf1.get_feature_names())
    s=s[s>0]
    s=pd.DataFrame(s)
    s.reset_index(inplace=True)
    for il in range(len(s)):
        tweet_series_1_tf.at[s['index'].iloc[il]]+=1

In [179]:
ts0=tweet_series_0_tf.copy()
ts0.sort_values(ascending=False, inplace=True)
ts0=pd.DataFrame(ts0)
ts0.reset_index(inplace=True)

ts1=tweet_series_1_tf.copy()
ts1.sort_values(ascending=False, inplace=True)
ts1=pd.DataFrame(ts1)
ts1.reset_index(inplace=True)

f=open('disaster_1000_words_tf.csv','w')
f.write('word,total_importance,importance_proportion')
for _ in range(1000):
    f.write(ts1.iloc[_]['index']+','+str(ts1.iloc[_]['count'])+','+str(ts1.iloc[_]['count']/len(ts1))[0:6]+',\n')
f.close()

f.write('word,total_importance,importance_proportion')
for _ in range(1000):
    f.write(ts0.iloc[_]['index']+','+str(ts0.iloc[_]['count'])+','+str(ts0.iloc[_]['count']/len(ts0))[0:6]+',\n')

f.close()

In [ ]:
from scipy.io import mmread

term_doc = mmread('../bbc/bbc.mtx')  # term-document matrix
doc_term = term_doc.T  # document-term matrix


from sklearn.decomposition import NMF

nmf2 = NMF(n_components=2)
doc_topic = nmf2.fit_transform(train.text)